In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import faiss
import time 
from dotenv import load_dotenv()

1.Loading the csv

In [ ]:
df=pd.read_csv('farhan11166/project/Documents/test csv/health_activity_data.csv')
df.columns = df.columns.str.lower()

2.Preparing the rag model.

In [ ]:
data=pd.read_csv('pandas_queries (1) (1).csv')
data.columns=['query','pandas_function_call']        
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
queries=[i for i in data['query']]
embeddings = model.encode(queries, convert_to_numpy=True)
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)
function_calls = [i for i in data['pandas_function_call']]
def retrieve_similar_queries(user_query, k=3):
    query_embedding = model.encode([user_query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    results = [(queries[i], function_calls[i]) for i in indices[0]]
    return results
def rag_pipeline(user_query):
    examples = retrieve_similar_queries(user_query)
    prompt = build_prompt(user_query, examples)
    result = query_grok_ai(prompt)
    return result    
def build_prompt(user_query, examples):
    prompt = """
You are an assistant that translates natural language into pandas code.
Use the examples to understand the pattern and write the correct function.
Return only the function in case of multiple function being availaible return onyl one of those.If two sperate functions are given instead of , use space.
Examples:
"""
    for q, a in examples:
        prompt += f"Q: {q}\nA: {a}\n\n"
    prompt += f"Now answer this:\nQ: {user_query}\nA:"
    return prompt

3.Methods for calling the API and returning valid uses

In [39]:
import os
from fastapi import FastAPI, UploadFile, Form
from fastapi.responses import JSONResponse
import pandas as pd
import io
import requests

app = FastAPI()

GROQ_API_KEY = os.getenv("API_KEY")
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
MODEL = "meta-llama/llama-4-scout-17b-16e-instruct" 

chat_history = [
    {"role": "system", "content":'''I have been given a csv file i will give you prompt regarding that data and then you must just return the 
    function to be used nothing else assume that the column exists else that the function is possible 
    dont give code just the function to be used for example if asked that what is total money spent return df['money'].sum() .Do not give functions in or give
    only one of in case .Do not give alternative functions give only one in case it can be done in multiple ways if choice is given between a more than 1 step function
    and a 1 step function always prefer one step function pass/Do not give alternative functions give only one incase mpre than one options are availaible.while using columnname match from one of columns present  '''}
]
column_info = f"The DataFrame contains the following columns: {', '.join(df.columns)}"
chat_history[0]["content"] += "\n\n" + column_info
first_five_rows = df.head(5)
data_summary = first_five_rows.to_json(orient='records')
chat_history.append({"role":"system","content":f"Here are the first five rows of my data: {data_summary}."})
datap=[]
def chat_with_groq(user_message,user_input):
    chat_history.append({"role": "user", "content": user_message})
    

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {GROQ_API_KEY}"
    }

    data = {
        "model": MODEL,
        "messages": chat_history
    }

    response = requests.post(GROQ_URL, headers=headers, json=data)
    response.raise_for_status()
    bot_message = response.json()["choices"][0]["message"]["content"]

    chat_history.append({"role": "assistant", "content": bot_message})
    datap.append({"query":user_input,"pandas_function_call":bot_message})
    return bot_message


4.Calling api returning functions for data and query and then generating the output on pc.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
while True:
    user_input = input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        break
    examples = retrieve_similar_queries(user_input)
    prompt = build_prompt(user_input, examples)
    response = chat_with_groq(prompt,user_input)
    
    try:
        
        l = [line.strip() for line in response.split(";")]
       
        l=l[len(l)-1]
       
        print(l)
        
        
        
        if "df" in l:
            alu=eval(response, {"__builtins__": {}}, {"df": df})
            print(alu)
            plt.show()
    except KeyError as e:
        print(f"❌ KeyError: Column {str(e)} does not exist.")
        datap.pop(len(datap)-1)
    except TypeError as e:
        print(f"❌ TypeError: {str(e)}")
        datap.pop(len(datap)-1)   
    except ValueError as e:
        print(f"❌ ValueError: {str(e)}")
        datap.pop(len(datap)-1)
    except AttributeError as e:
        print(f"❌ AttributeError: {str(e)}")
        datap.pop(len(datap)-1)
    except SyntaxError as e:
        print(f"❌ SyntaxError: {str(e)}   {response}")
        datap.pop(len(datap)-1)
    except NameError as e:
        print(f"❌ NameError: {str(e)}")
        datap.pop(len(datap)-1)
        

    except Exception as e:
        print(f"❌ Unexpected Error: {str(e)}")
        datap.pop(len(datap)-1)
    time.sleep(1)    
      


You:  calculate correlation of height and weight


df[['height_cm', 'weight_kg']].corr().iloc[0,1]
-0.05037718176212504


You:   calculate correlation of height and weight in women


df[df['gender'] == 'Female'][['height_cm', 'weight_kg']].corr().iloc[0,1]
-0.09145422851594309


You:  calculate correlation of height and weight in men


df[df['gender'] == 'Male'][['height_cm', 'weight_kg']].corr().iloc[0,1]
-0.01441903599585993


You:  calculate correlation of diabetice people and wieght


df[df['diabetic'] == 'Yes'][['weight_kg']].std()
weight_kg    19.271587
dtype: float64


You:  exit


5.Saving it in the data used for rag model hence improving the odel for future use.

In [ ]:
import os 
fer=pd.DataFrame(datap)
filep='pandas_queries (1) (1).csv'
write_header = not os.path.exists(filep)

fer.to_csv(filep, mode='a', index=False, header=write_header)
data=pd.read_csv('pandas_queries (1) (1).csv')
final=data.drop_duplicates()
final.to_csv(filep,index=False,header=write_header)